# Introduction
<hr style = "border:2px solid black" ></hr>


**What?** How to get the derivatives from XGBoost



# Import modules
<hr style = "border:2px solid black" ></hr>

In [1]:
import numpy as np
import xgboost as xgb
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix

/opt/anaconda3/envs/trainingAI/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


# Keep track of the dradient updates
<hr style = "border:2px solid black" ></hr>


- To keep track of the gradient updates in each iteration, you would need to **expose the training loop** in python (rather than have it execute internally in the C++ implementation), and provide custom gradient and hessian implementations. 
- For many of the standard loss functions, e.g., squared loss, logistic loss, this is quite simple and not hard to find in standard references. 
- Here's an example showing how to expose the training loop for logistic regression. 



# Create synthetic dataset
<hr style = "border:2px solid black" ></hr>

In [2]:
# Build a toy dataset.
X, Y = make_classification(n_samples=1000, n_features=5, n_redundant=0, n_informative=3,
                           random_state=1, n_clusters_per_class=1)

# Define a loss function and its derivatives
<hr style = "border:2px solid black" ></hr>

In [3]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))


def logregobj(preds, dtrain):
    """log likelihood loss"""
    labels = dtrain.get_label()
    preds = sigmoid(preds)
    grad = preds - labels
    hess = preds * (1.0-preds)
    return grad, hess

# Create an XGBoost model
<hr style = "border:2px solid black" ></hr>

In [4]:
# Instantiate a Booster object to do the heavy lifting
dtrain = xgb.DMatrix(X, label=Y)
params = {'max_depth': 2, 'eta': 1}
num_round = 2
model = xgb.Booster(params, [dtrain])

# How to get the gradients
<hr style = "border:2px solid black" ></hr>


- To keep track of the gradient updates in each iteration, you would need to **expose the training loop** in python (rather than have it execute internally in the C++ implementation), and provide custom gradient and hessian implementations. 
- This means creating a loop manually in python rather than using the `fit()` method!
    


In [5]:
# Run 10 boosting iterations
# g and h can be monitored for gradient statistics
for _ in range(10):
    pred = model.predict(dtrain)
    g, h = logregobj(pred, dtrain)
    model.boost(dtrain, g, h)

In [6]:
g.shape

(1000,)

In [7]:
h.shape

(1000,)

# Evaluate predictions 
<hr style = "border:2px solid black" ></hr>

In [8]:
yhat = model.predict(dtrain)
yhat = 1.0 / (1.0 + np.exp(-yhat))
yhat_labels = np.round(yhat)
confusion_matrix(Y, yhat_labels)

array([[498,   4],
       [  7, 491]])

# References
<hr style = "border:2px solid black" ></hr>


- https://stackoverflow.com/questions/44916391/how-to-get-or-see-xgboosts-gradient-statistics-value

